In [75]:
include("dm.jl")

infer (generic function with 6 methods)

# Factor example

In [76]:
X = Variable(:x, 2)
Y = Variable(:y, 2)
Z = Variable(:z, 2)
φ = Factor([X, Y, Z], FactorTable(
    (x=1, y=1, z=1) => 0.08, (x=1, y=1, z=2) => 0.31,
    (x=1, y=2, z=1) => 0.09, (x=1, y=2, z=2) => 0.37,
    (x=2, y=1, z=1) => 0.01, (x=2, y=1, z=2) => 0.05,
    (x=2, y=2, z=1) => 0.02, (x=2, y=2, z=2) => 0.07,
));

In [77]:
function print_factor(φ::Factor)
    for (a,p) in φ.table
        println(a, " with probability ", p)
    end 
end  

print_factor(φ)

Dict(:y => 2, :z => 2, :x => 1) with probability 0.37
Dict(:y => 1, :z => 2, :x => 1) with probability 0.31
Dict(:y => 2, :z => 1, :x => 2) with probability 0.02
Dict(:y => 1, :z => 1, :x => 1) with probability 0.08
Dict(:y => 1, :z => 2, :x => 2) with probability 0.05
Dict(:y => 2, :z => 2, :x => 2) with probability 0.07
Dict(:y => 2, :z => 1, :x => 1) with probability 0.09
Dict(:y => 1, :z => 1, :x => 2) with probability 0.01


In [78]:
names = variablenames(φ)

3-element Vector{Symbol}:
 :x
 :y
 :z

In [79]:
a = Assignment((x=1,y=2,z=1))

Dict{Symbol, Int64} with 3 entries:
  :y => 2
  :z => 1
  :x => 1

In [80]:
φ.table[a]

0.09

In [81]:
select(a,[:z])

Dict{Symbol, Int64} with 1 entry:
  :z => 1

# Example 2.5: satellite-monitoring problem

In [82]:
B = Variable(:b, 2); S = Variable(:s, 2)
E = Variable(:e, 2)
D = Variable(:d, 2); C = Variable(:c, 2)

vars = [B, S, E, D, C]

factors = [
    Factor([B], FactorTable((b=1,) => 0.99, (b=2,) => 0.01)),
    Factor([S], FactorTable((s=1,) => 0.98, (s=2,) => 0.02)),
    Factor([E,B,S], FactorTable(
        (e=1,b=1,s=1) => 0.90, (e=1,b=1,s=2) => 0.04,
        (e=1,b=2,s=1) => 0.05, (e=1,b=2,s=2) => 0.01,
        (e=2,b=1,s=1) => 0.10, (e=2,b=1,s=2) => 0.96,
        (e=2,b=2,s=1) => 0.95, (e=2,b=2,s=2) => 0.99)),
    Factor([D, E], FactorTable(
        (d=1,e=1) => 0.96, (d=1,e=2) => 0.03,
        (d=2,e=1) => 0.04, (d=2,e=2) => 0.97)),
    Factor([C, E], FactorTable(
        (c=1,e=1) => 0.98, (c=1,e=2) => 0.01,
        (c=2,e=1) => 0.02, (c=2,e=2) => 0.99))
]

graph = SimpleDiGraph(5)
add_edge!(graph, 1, 3); add_edge!(graph, 2, 3)
add_edge!(graph, 3, 4); add_edge!(graph, 3, 5)

bn = BayesianNetwork(vars, factors, graph)

for factor in bn.factors
    print_factor(factor)
end

Dict(:b => 1) with probability 0.99
Dict(:b => 2) with probability 0.01
Dict(:s => 2) with probability 0.02
Dict(:s => 1) with probability 0.98
Dict(:b => 1, :s => 1, :e => 1) with probability 0.9
Dict(:b => 1, :s => 1, :e => 2) with probability 0.1
Dict(:b => 2, :s => 2, :e => 2) with probability 0.99
Dict(:b => 1, :s => 2, :e => 2) with probability 0.96
Dict(:b => 2, :s => 2, :e => 1) with probability 0.01
Dict(:b => 1, :s => 2, :e => 1) with probability 0.04
Dict(:b => 2, :s => 1, :e => 1) with probability 0.05
Dict(:b => 2, :s => 1, :e => 2) with probability 0.95
Dict(:d => 2, :e => 1) with probability 0.04
Dict(:d => 2, :e => 2) with probability 0.97
Dict(:d => 1, :e => 2) with probability 0.03
Dict(:d => 1, :e => 1) with probability 0.96
Dict(:e => 2, :c => 1) with probability 0.01
Dict(:e => 1, :c => 1) with probability 0.98
Dict(:e => 1, :c => 2) with probability 0.02
Dict(:e => 2, :c => 2) with probability 0.99


In [83]:
# Testing the calculation of assignment probabilities
a = Assignment((b=1, s=1, e=1, d=1,c=1)) # 2 = fail
probability(bn, a)


0.8214877439999999

In [98]:
# Testing factor product
a = factors[1]
b = factors[3]
c = a*b
c.table

Dict{Dict{Symbol, Int64}, Float64} with 8 entries:
  Dict(:b=>1, :s=>1, :e=>1) => 0.891
  Dict(:b=>1, :s=>1, :e=>2) => 0.099
  Dict(:b=>2, :s=>2, :e=>2) => 0.0099
  Dict(:b=>1, :s=>2, :e=>2) => 0.9504
  Dict(:b=>2, :s=>2, :e=>1) => 0.0001
  Dict(:b=>1, :s=>2, :e=>1) => 0.0396
  Dict(:b=>2, :s=>1, :e=>1) => 0.0005
  Dict(:b=>2, :s=>1, :e=>2) => 0.0095

In [85]:
# Testing marginalise
m = normalize!(marginalize(factors[3], :b))
m.table

Dict{Dict{Symbol, Int64}, Float64} with 4 entries:
  Dict(:s=>2, :e=>1) => 0.0125
  Dict(:s=>2, :e=>2) => 0.4875
  Dict(:s=>1, :e=>1) => 0.2375
  Dict(:s=>1, :e=>2) => 0.2625

In [86]:
topological_sort_by_dfs(bn.graph) # This is the correct function, not topological_sort()

5-element Vector{Int64}:
 2
 1
 3
 5
 4

In [87]:
# Random sampling from a bn.
for i in 1:10
    println(rand(bn))
end

Dict(:b => 1, :d => 2, :s => 2, :e => 2, :c => 2)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 2, :s => 1, :e => 2, :c => 2)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)
Dict(:b => 1, :d => 1, :s => 1, :e => 1, :c => 1)


In [88]:
# Calculating the conditional prob. of a variable, given an assignment. 
evidence = (s=2, b=2, d=1) # A bit silly, but evidence must contain the variable of interest.ß
a = Assignment(evidence)
b = blanket(bn, a, 4) 

Factor(Variable[Variable(:d, 2), Variable(:e, 2)], Dict(Dict(:d => 2, :e => 1) => 0.02, Dict(:d => 2, :e => 2) => 0.485, Dict(:d => 1, :e => 2) => 0.015, Dict(:d => 1, :e => 1) => 0.48))

In [89]:
method = VariableElimination([1,2,3,4,5])
@time infer(method, bn, [:e], (b=2, s=1) ) #evidence is given as a named tuple

  0.411456 seconds (627.79 k allocations: 32.498 MiB, 3.53% gc time, 99.64% compilation time)


Factor(Variable[Variable(:e, 2)], Dict(Dict(:e => 2) => 0.95, Dict(:e => 1) => 0.05))

In [90]:
method = DirectSampling(1000)
@time infer(method, bn, [:e], (b=2, s=1) )

  0.079411 seconds (211.50 k allocations: 16.011 MiB, 93.23% compilation time)


Factor(Variable[Variable(:e, 2)], Dict(Dict(:e => 2) => 1.0))

In [91]:
method = LikelihoodWeightedSampling(1000)
@time infer(method, bn, [:e], (b=2, s=1) )

  0.079798 seconds (125.72 k allocations: 11.868 MiB, 92.61% compilation time)


Factor(Variable[Variable(:e, 2)], Dict(Dict(:e => 2) => 0.9330000000000007, Dict(:e => 1) => 0.06699999999999932))

In [92]:
method = GibbsSampling(1000, 5, 10, [1,2,3,4,5])
@time infer(method, bn, [:e], (b=2, s=1) )

Dict(:b => 2, :d => 1, :s => 1, :e => 1, :c => 1)
  0.320539 seconds (2.23 M allocations: 230.129 MiB, 10.45% gc time, 29.83% compilation time)


Factor(Variable[Variable(:e, 2)], Dict(Dict(:e => 2) => 0.936, Dict(:e => 1) => 0.064))

In [93]:
D = MvNormal([0.0,1.0],[3.0 1.0; 1.0 2.0])
infer(D, [1], [2], [2.0])

FullNormal(
dim: 1
μ: [0.5]
Σ: [2.5;;]
)
